In [1]:
import rioxarray as rxr
import os
from rasterio.plot import show
from PIL import Image
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd
import rasterio.features
from shapely.geometry import shape
import pandas as pd
import time

In [2]:
# retreive some infos on fully contained segments
contained_file_50 = 'data/contained_segments_50.txt'
contained_file_0001 = 'data/contained_segments_0001.txt'
contained_file_100 = 'data/contained_segments_100.txt'

In [3]:
folder = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_output"

list_bands = ['02', '03', '04', '05', '06', '07', '08', '8A', '11', '12']

- Satellite raster data (Sentinel 2)
	- 10 spectral bands, one per file (s2_2020_B02.tif, s2_2020_B03.tif, etc...)
	- Each file s2_2020_BXX.tif: contains an array of size (73, 4513, 5183)=(nb. dates, height, width).

In [4]:
def load_data_from_file(contained_file):
    # load the data
    with open(contained_file) as f:
        lines = f.readlines()
        # reteive line infos
        segment_id = []
        polygon_id = []
        class_id = []
        pixels = []
        for line in lines:
            # retrieve segment id
            segment_id.append(line.split(',')[0].split(':')[1].strip())
            # retrieve polygon id
            polygon_id.append(line.split(',')[1].split(':')[1].strip())
            # retrieve class id
            class_id.append(line.split(',')[2].split(':')[1].strip())
            # retrieve pixels
            pixel_list_str = line.split(':')[4]
            pixel_list_str = pixel_list_str[3:-3]
            pixel_list = list(pixel_list_str.split('], ['))
            pixel_list = [pixel.split(', ') for pixel in pixel_list]
            pixel_list = [[int(pixel[0]), int(pixel[1])] for pixel in pixel_list]
            pixels.append(pixel_list)
        
        # create a dataframe
        df = pd.DataFrame()
        df['segment_id'] = segment_id
        df['polygon_id'] = polygon_id
        df['class_id'] = class_id
        df['pixels'] = pixels
        
        return df

In [5]:
dataframe_segment = load_data_from_file(contained_file_50)

In [19]:
# show mean lenth of segments' pixels
mean_lenth = []
for pixels in dataframe_segment['pixels']:
    mean_lenth.append(len(pixels))
print("mean lenth of segments' pixels")
print(np.mean(mean_lenth))

# show mean number of segments per polygon
print("mean number of segments per polygon")
print(len(dataframe_segment['segment_id'].unique())/len(dataframe_segment['polygon_id'].unique()))

# show number of polygons
print("number of polygons")
print(len(dataframe_segment['polygon_id'].unique()))

# show total number of pixels
print("total number of pixels")
print(len(dataframe_segment['pixels'].sum()))

mean lenth of segments' pixels
16.178727841501566
mean number of segments per polygon
6.382157123834887
number of polygons
751
total number of pixels
77577


In [ ]:
start = 1 # always 1
acquisition = 73 # 73
folder = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_gathered_output"
print(f"folder: {folder}")

for band in list_bands:
    file_path = os.path.join(folder, f'normalized_s2_2020_B{band}.tif')
    
    for period in range(start, acquisition+1):
        with rasterio.open(file_path) as src:
            # check if all acquisitions exist as source in the file
            print(src.count)
